# Single Perceptron Reinforcement Learning-Based Parameter Estimation

__Goal__: Simplest possible proof of concept. Create a policy $\pi$ that can reliably converge a new perceptron to a pre-trained perceptron's weights/behavior based on predictions from a dataset. Discrete/tabular Q-learning will be used for simplicity.

## Algorithm design

__Perceptron__: $\hat y = \vec w^T \vec x$ for $\vec w, \vec x \in \mathcal{R}^{d}$.

__Actions__: $a\in \mathcal{A} = \mathcal{\{-1, 0, +1\}}^d$. 

__State__: For simplicity, state $s\in \mathcal S$ for each synapse: 

1. Previous action $\in \{-1, 0, +1\}$.
2. Previous reward $\in \{-1, 0, +1\}$.

State will be extended $h$ iterations back in time to provide model with additional data. The value of $h$ will likely need to be proportional to the dimensionality $d$ of the data space. 

__Training__: $[\vec w_{k+1}]_i = \epsilon \pi([s_k]_i) + [\vec w_{k+1}]_i$
 - Where $\epsilon$ is the neuron learning rate. 

## Implementation

Perceptron, actions, annd training can be implemented in a straightforward way with the `LinearAlgebra` library. State can be a simple rolling list for each synapse. 

Tabular policy can be a multidimensional array where $\pi[\text{previous action}][\text{previous reward}][...]$ can be directly indexed.

To generate target perceptron and training data, we can uniformly generate weight vector entries and datapoints in a given range. 

 - $\vec w_i \sim \text{Uniform}[\pm w_{range}]$; $w_{range} = 10$
 - $\vec x_i \sim \text{Uniform}[\pm x_{range}]$; $x_{range} = 20$ (except for entry $x_1$, which is a bias entry). 

To reduce the chance of runaway arguments, we can adjust the update rule in __Training__ such that the trained model's vector entry $w_i$ cannot exceed $\pm 1.5 w_{range}$


## Q-learning Formulae
$$Q(s, a) = R_{t+1} + \gamma \max_{a'} Q(s',a')$$
$$Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha[R_{t+1} + \gamma \max_{a'}(Q(s', a') - Q(s_t, a_t))$$

__Q-Learning Parameters__

 - $\alpha = 0.01$: Learning rate 
 - $\gamma = 0.8$: Future reward discount factor

## Lessons/Resulting Ideas

 - Would be interesting to treat each synapse weight as its own time-series (ML?) estimation problem.
 - Should _synapses_ be the fundamental unit rather than neurons? Biologically-realistic neurons have an incredibly large number of synapses, and a policy that non-linearly incorporates all synapses weights/changes would be extremely difficult computationally.
 - Some sort of actual regularization is going to become necessary very quickly.
 - In the actual brain, there's probably some optimal proportionality between number of highly variable synapses and "memory length" for each neuron...


In [21]:
# IMPORT BOX #
using LinearAlgebra
using Plots
# using Plotly
using Random

import JSON

# plotly(size=(500,500))

In [22]:
# CONSTANTS #
DIM = 3 # Dimensionality of weight vector (data has d-1 dimensions)
W_RANGE = 10 # Range for the target vector (~uniform(+/-))
X_RANGE = 20 # Range for values in training data (~uniform(+/-))
N = 200 # Number of datapoints in a given batch of generated data.
N_HIST = 2 # Number of `action` and `rewards` in the history buffer.
# ϵ = 0.1 # Weight update rate
ϵ = 0.01 # Weight update rate
α = 0.1 # Q-learning rate
γ = 0.8 # Future reward discount factor

0.8

## Caching Functions
 - [ ] JSON file entries:
     - [ ] Include `loss` per iteration.
     - [ ] Include `target_perceptron` parameters.
     - [ ] Include `generated_perceptron` parameters.
     - [ ] Include `generated_policy`.
     - [ ] Include `datapoints` x.
     - [ ] Include boolean `train_policy`.
 - [ ] `/results/final/linear_2_adaptive.json`
 - [ ] `/results/final/linear_2_static.json`

In [23]:
function cache_01(destination; losses, target_perceptron, generated_perceptron, policy, 
                  datapoints, train_policy, alpha, epsilon, gamma, explore_prob)
    """
    Function to cache `loss[]`, `target perceptron`, `generated perceptron`, 
    `generated_policy`, `x`, `train_policy`.
    """
    
    output_dict = Dict{String, Any}(
        "losses" => losses,
        "target_perceptron" => target_perceptron,
        "generated_perceptron" => generated_perceptron,
        "policy" => policy,
        "datapoints" => datapoints,
        "train_policy" => train_policy,
        "alpha" => alpha,
        "epsilon" => epsilon,
        "gamma" => gamma,
        "explore_prob" => explore_prob
    )
    
    output_string = JSON.json(output_dict)

	open(destination, "w") do f
	    write(f, output_string)
	end
    
    println("Caching successful.")
end

cache_01 (generic function with 1 method)

## Part I: Setting Up Mathematical Objects

 - Perceptron struct
 - Policy struct
 - Predict() function
 - Random perceptron + data generation
 - Plotting functions

In [24]:
mutable struct perceptron 
    w::Array{Float64, 1} # 2-dimensional arra of float64's
end

In [25]:
struct policy
    Q::Array{Float64, N_HIST*2+1} # for each history round we have `actions` and `rewards`
end

"""
Q[action][previous action][previous reward][next previous action][next previous reward][...]
is the expected value of the `action` given the previous data.

The number of `previous action`/`previous reward` pairs is given by `N_HIST`. 

In order to index the current `action` and all previous `action/reward pairs`, we require
an array with dimensionality `N_HIST*2 + 1`.

In this version, each of `action` and `reward` can take on 3 values. Therefore, the 
policy tensor is 3*3*3*3*...
"""

"Q[action][previous action][previous reward][next previous action][next previous reward][...]\nis the expected value of the `action` given the previous data.\n\nThe number of `previous action`/`previous reward` pairs is given by `N_HIST`. \n\nIn order to index the current `action` and all previous `action/reward pairs`, we require\nan array with dimensionality `N_HIST*2 + 1`.\n\nIn this version, each of `action` and `reward` can take on 3 values. Therefore, the \npolicy tensor is 3*3*3*3*...\n"

In [26]:
function new_policy()
    size = convert(Array{Int,1}, ones(N_HIST*2+1)*3)
    size_tuple = tuple(size...)
    pol = policy(zeros( size_tuple ))
    return pol
end

new_policy (generic function with 1 method)

In [27]:
function predict(p::perceptron, x)
    # Where columns of x are datapoints
    return transpose(x)*p.w
end

predict (generic function with 1 method)

In [28]:
function generate_perceptron(DIM)
    w = rand(Float64, DIM);
    w*=2;
    w =w.-1;
    w*=W_RANGE;
    p = perceptron(w);
    return p;
end

generate_perceptron (generic function with 1 method)

In [29]:
function generate_data(DIM, X_RANGE, N)
    x = rand(Float64, (DIM, N));
    x *= 2;
    x = x .- 1;
    x *= X_RANGE;
    x[1,:] .= 1; # Bias term
    return x;
end

generate_data (generic function with 1 method)

In [31]:
"""PLOTTING TEST CODE"""

# Perceptron generation test
percep = generate_perceptron(DIM)

# Data generation test
x1 = generate_data(DIM, X_RANGE, N) 
x2 = generate_data(DIM, X_RANGE, N)

# Putting datasets thru perceptron
y_pred1 = predict(percep, x1);
y_pred2 = predict(percep, x2);

# Scatter plot of each
Plots.scatter(x1[2,:], x1[3,:], y_pred1)
Plots.scatter!(x2[2,:], x2[3,:], y_pred2)
Plots.title!("Test of single Perceptron")

<!DOCTYPE html>
 
 
 Plots.jl

## Part II: Building Training Loop

 - Initialize target perceptron and training perceptron.
     - Initialize policy
 - Loss (reward) function (args: perceptron1, perceptron2, dataset x)
 - Implement Q-learning formulae:
     - Keep previous actions and rewards in a running list. 
     - At the end of the loop, implement the update rule. 
     - 80% of the time, take the "optimal" action.
     - Next: Implement exploration decay.
     
     
---
$$Q(s, a) = \mathbb E [R_{t+1} + \gamma R_{t+1} + \gamma^2 R_{t+2} + \dots]$$
$$Q(s, a) = R_{t+1} + \gamma \max_{a'} Q(s',a')$$
$$Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha[R_{t+1} + \gamma \max_{a'}(Q(s', a') - Q(s_t, a_t))$$

In [32]:
function loss(percep1, percep2, x)
    pred1 = predict(percep1, x);
    pred2 = predict(percep2, x);
    diff = pred1-pred2;
    return norm(diff)/length(pred1);
end

loss (generic function with 1 method)

In [33]:
"""Testing loss function"""
# Perceptron generation test
percep1 = generate_perceptron(DIM)
percep2 = generate_perceptron(DIM)

# Data generation test
x1 = generate_data(DIM, X_RANGE, N) 

percep2.w = copy(percep1.w)
percep2.w[1]+=10
percep2.w[2]+=5
println(percep1)
println(percep2)

loss(percep1, percep2, x1)

perceptron([-1.8176313882027406, -2.50624077175023, 2.7433165701261997])
perceptron([8.182368611797258, 2.49375922824977, 2.7433165701261997])


4.206026519796725

In [34]:
"""Generating train/target perceptrons, policy, data"""
percep = generate_perceptron(DIM)
target = generate_perceptron(DIM)

pol = new_policy();

x = generate_data(DIM, X_RANGE, N);

In [35]:
num_iterations = 10000;

actions = zeros(DIM, N_HIST).+2 # For DIM synapses we store their actions.
                             # Starting with N_HIST actions set to 0.
rewards = zeros(N_HIST).+2 # Rewards history is instantiated with N_HIST zeros.
losses = zeros(N_HIST).+loss(percep, target, x)
exploration_probability = 0.5;

for i = 1:num_iterations
    # Adding another entry to actions, rewards
    actions = [actions zeros(DIM)]
    append!(rewards, 0)
#     break
    # Iterating through each synapse and taking an action (inc/dec/same).
    for j = 1:DIM
        addr = [actions[j, end-N_HIST:end-1]; rewards[end-N_HIST:end-1]]
        addr = convert(Array{Int64,1}, addr)
#         println("Type of addr: ", typeof(addr))
#         break
        a = 0;
        if rand() < exploration_probability # Currently we do no exploration for debugging TODO REMOVE
            a = rand([1, 2, 3]); # Chose a random move
        else
#             println("ACCESSING")
            a = argmax(pol.Q[:,addr...]); # Choose the "optimal" move given the 
                                          # Past actions and rewards
        end
        
        # We have now chosen our action.
        actions[j,end] = a;
        
        # Taking the action
        percep.w[j] += (a-2)*ϵ;
    end
    # Calculating reward for this iteration through all synapses.
    new_loss = loss(percep, target, x);
    R = 0;
    if(new_loss < losses[end])
        R = 3;
    else
        R = 1;
    end
#     println("Old loss: ", losses[end])
#     println("New loss: ", new_loss)
#     println("Reward: ", R)
    
    rewards[end] = R;
    append!(losses, new_loss)
    
    # Applying update rule for Q values on each synapse
    for j = 1:DIM
        addr_old = [actions[j, end-N_HIST:end-1]; rewards[end-N_HIST:end-1]]
        addr_new = [actions[j, end-N_HIST+1:end]; rewards[end-N_HIST+1:end]]
        
        addr_old = convert(Array{Int64,1}, addr_old)
        addr_new = convert(Array{Int64,1}, addr_new)
        
        max_future_term = γ*maximum(pol.Q[:,addr_new...]); 
        
        cur_term = pol.Q[ convert(Int64,actions[j,end]) , addr_old...];
        
        pol.Q[ convert(Int64,actions[j,end]) , addr_old...] = cur_term + α*( (R-2) + max_future_term - cur_term); 
    end
end

In [36]:
rand([1, 2, 4])

4

In [37]:
# Scatter plot of each
println("Number of iterations: ", num_iterations)
println("Loss: ", losses[end])
println("WITH simultaneous perceptron/policy training")
Plots.scatter(x[2,:], x[3,:], predict(percep, x))
Plots.scatter!(x[2,:], x[3,:], predict(target, x))
Plots.title!("Trained vs. Target Perceptron")

Number of iterations: 10000
Loss: 0.02619610611649706
WITH simultaneous perceptron/policy training


<!DOCTYPE html>
 
 
 Plots.jl

In [38]:
Plots.plot(losses)
Plots.title!("Losses over Time (Simult. Training)")

<!DOCTYPE html>
 
 
 Plots.jl

### Caching Adaptive 2D (DONE)

In [39]:
cache_01("../results/final/linear_2_adaptive.json"; losses=losses, target_perceptron=target.w, 
         generated_perceptron=percep.w, policy=pol, datapoints=x, train_policy=true, alpha=α, epsilon=ϵ, gamma=γ,
         explore_prob = exploration_probability)

Caching successful.


## Testing Generalizability
Applying learned perceptron training policy to new perceptron.

In [40]:
"""Generating train/target perceptrons, policy, data"""
percep = generate_perceptron(DIM)
target = generate_perceptron(DIM)

# pol = new_policy(); # STICKING TO OLD POLICYs

x = generate_data(DIM, X_RANGE, N);

In [41]:
num_iterations = 10_000;

actions = zeros(DIM, N_HIST).+2 # For DIM synapses we store their actions.
                             # Starting with N_HIST actions set to 0.
rewards = zeros(N_HIST).+2 # Rewards history is instantiated with N_HIST zeros.
losses = zeros(N_HIST).+loss(percep, target, x)
exploration_probability = 0.5;

train_policy = false;

for i = 1:num_iterations
    # Adding another entry to actions, rewards
    actions = [actions zeros(DIM)]
    append!(rewards, 0)
#     break
    # Iterating through each synapse and taking an action (inc/dec/same).
    for j = 1:DIM
        addr = [actions[j, end-N_HIST:end-1]; rewards[end-N_HIST:end-1]]
        addr = convert(Array{Int64,1}, addr)
#         println("Type of addr: ", typeof(addr))
#         break
        a = 0;
        if rand() < exploration_probability # Currently we do no exploration for debugging TODO REMOVE
            a = rand([1, 2, 3]); # Chose a random move
        else
#             println("ACCESSING")
            a = argmax(pol.Q[:,addr...]); # Choose the "optimal" move given the 
                                          # Past actions and rewards
        end
        
        # We have now chosen our action.
        actions[j,end] = a;
        
        # Taking the action
        percep.w[j] += (a-2)*ϵ;
    end
    # Calculating reward for this iteration through all synapses.
    new_loss = loss(percep, target, x);
    R = 0;
    if(new_loss < losses[end])
        R = 3;
    else
        R = 1;
    end
#     println("Old loss: ", losses[end])
#     println("New loss: ", new_loss)
#     println("Reward: ", R)
    
    rewards[end] = R;
    append!(losses, new_loss)
    
    # Applying update rule for Q values on each synapse
    if train_policy
        for j = 1:DIM
            addr_old = [actions[j, end-N_HIST:end-1]; rewards[end-N_HIST:end-1]]
            addr_new = [actions[j, end-N_HIST+1:end]; rewards[end-N_HIST+1:end]]

            addr_old = convert(Array{Int64,1}, addr_old)
            addr_new = convert(Array{Int64,1}, addr_new)

            max_future_term = γ*maximum(pol.Q[:,addr_new...]); 

            cur_term = pol.Q[ convert(Int64,actions[j,end]) , addr_old...];

            pol.Q[ convert(Int64,actions[j,end]) , addr_old...] = cur_term + α*( (R-2) + max_future_term - cur_term); 
        end
    else
#         println("NOT UPDATING POLICY!!");
    end
end

In [44]:
Plots.plot(losses)
Plots.title!("Loss over Time (Applying Learned Policy)")

<!DOCTYPE html>
 
 
 Plots.jl

In [45]:
# Scatter plot of each
println("Number of iterations: ", num_iterations)
println("Loss: ", losses[end])
println("WITH LEARNED learning policy")
Plots.scatter(x[2,:], x[3,:], predict(percep, x))
Plots.scatter!(x[2,:], x[3,:], predict(target, x))
Plots.title!("Trained vs. Target Perceptron")


Number of iterations: 10000
Loss: 0.10044832151280735
WITH LEARNED learning policy


<!DOCTYPE html>
 
 
 Plots.jl

### Caching Static Policy Application (DONE)

In [46]:
cache_01("../results/final/linear_2_static.json"; losses=losses, target_perceptron=target.w, 
         generated_perceptron=percep.w, policy=pol, datapoints=x, train_policy=false, alpha=α, epsilon=ϵ, gamma=γ,
         explore_prob = exploration_probability)


# cache_01("../results/final/linear_2_adaptive.json"; losses=losses, target_perceptron=target.w, 
#          generated_perceptron=percep.w, policy=pol, datapoints=x, train_policy=true, alpha=α, epsilon=ϵ, gamma=γ,
#          )

Caching successful.
